In [30]:
from PIL import Image
import matplotlib.pyplot as plt
from networkx import DiGraph
from scipy.ndimage.measurements import center_of_mass
import numpy as np
from scipy.ndimage.measurements import find_objects

def prepImage(image='../test_cases/e2.png'):
    img = Image.open(image).convert('L')
    img = np.asarray(img)
    img2 = np.copy(img)
    
    if 'e4' in image:
        img2[np.where(img == 195)] = 0.0
        img2[np.where(img == 229)] = 3.0
        img2[np.where(img == 37)] = 2.0
        img2[np.where(img == 100)] = 1.0
        
        return img2
    else:
        img2[np.where(np.logical_and(np.greater(img,0),np.less_equal(img,168)))] = 1.0
        img2[np.where(img == 255)] = 2.0
    
    return img2

class Formatter(object):
    '''
    Use like:
        fig, ax = plt.subplots()
        im = ax.imshow(img, cmap='gray')
        ax.format_coord = Formatter(im)
    '''
    %matplotlib
    def __init__(self, im):
        self.im = im
    def __call__(self, x, y):
        z = self.im.get_array()[int(y), int(x)]
        return 'x={:.01f}, y={:.01f}, z={:.01f}'.format(x, y, z)
    
    

Using matplotlib backend: TkAgg


In [31]:
img = prepImage(image='../test_cases/e4.png')
fig, ax = plt.subplots()
im = ax.imshow(img)
ax.format_coord = Formatter(im)
plt.show()

# Check if the test cases are plausible or not

In [32]:
testcase = np.array(
    [[3,3,3,3],
     [3,2,2,3],
     [1,2,2,2],
     [1,2,2,2],
     [2,1,1,2],
     [3,1,1,3],
     [3,3,3,3]]
)

g = DiGraph()

# self links
g.add_edge(0,0)
g.add_edge(1,1)
g.add_edge(2,2)
g.add_edge(3,3)

g.add_edge(1,0)
g.add_edge(1,3)
g.add_edge(1,2)
g.add_edge(2,0)
g.add_edge(2,3)
g.add_edge(3,0)
plt.imshow(testcase)
plt.show()

In [33]:
from scipy.ndimage.measurements import find_objects

In [34]:
testcase= img

In [35]:
def maskExcept(im, e=1):
    tm = np.array(im)
    tm[np.where(tm != e)] = 0
    return tm

core = maskExcept(testcase, e=1) # get only the necrotic core

if np.count_nonzero(core) == 0:
    print('No necrotic core found, reverting to enhancing!')
    core = maskExcept(testcase, e=2) # get the enhancing core
    
comr, comc = center_of_mass(core)
comr = int(comr)
comc = int(comc)
print(comr, comc)

names = {
    1: 'necrotic',
    2: 'enhancing',
    3: 'edema'
}

# start the scanning process
switch = []
ranges = {'up': range(int(comr), -1, -1), 
          'down': range(int(comr), np.shape(testcase)[0], 1), 
          'left': range(int(comc), -1, -1), 
          'right': range(int(comc), np.shape(testcase)[1], 1)
         }

for direction in ranges:
    prev = None
    flag = 1
    for coord in ranges[direction]:
        if direction == 'up' or direction == 'down':
            r = coord
            c = comc
        else:
            r = comr
            c = coord

        if prev == None:
            prev = testcase[r,c]
        else:
            if g.has_edge(prev, testcase[r,c]):
                prev = testcase[r,c]
                continue
            else:
                print('invalid!')
                flag = -1
                break
            
    if flag != -1:
        print('Passed the {} direction test!'.format(direction))
    else:
        print('Failed the {} direction test!'.format(direction))

(9, 17)
Passed the down direction test!
Passed the right direction test!
Passed the up direction test!
Passed the left direction test!
